In [4]:
import cv2
import math
import os
import shutil
import numpy as np
import imageio
print np.__version__
print cv2.__version__

1.14.5
3.4.1


In [245]:
def get_list_of_imagesPath(folder):
    """
    create a list of all images' paths  for all subfolders in the folder
    """

    fd_list = []
    
    if not all_dir(folder):
        return sorted(map(lambda x: os.path.join(folder, x), os.listdir(folder)))
    
    for fd in os.listdir(folder):
        path = os.path.join(os.getcwd(), folder, fd)
        fd_list += get_list_of_imagesPath(path)

    return fd_list 

In [246]:
def all_dir(folder):
    """
    serve to check all filenames in folder is dir
    """
    flag = True
    for fd in os.listdir(folder):
        flag = True and os.path.isdir(os.path.join(folder,fd))
        
    return flag

In [289]:
def copy_files_to_folder(list_files, folder):
    """
    copy the list of files in the folder
    """
    # check the existing of the folder if not make
    if not os.path.exists(os.path.join(os.getcwd(),folder)):
        os.makedirs(os.path.join(os.getcwd(), folder))
    
    for f in list_files:
        shutil.copyfile(f , os.path.join(os.getcwd(), folder, f.split('/')[-1]))
    

In [290]:
def create_Folders_for_train_test(folder, test_size=0.15):
    """
    seperate images on train and test foldes 
    list images - list of image' paths 
    test_size is the size of the test folder 
    """
    from sklearn.cross_validation import train_test_split
    
    list_images = np.array(get_list_of_imagesPath(folder))
    y = np.vectorize(lambda x: x.split('/')[-3])(list_images)
    
    images_train, images_test, y_train, y_test = train_test_split(list_images, y, stratify=y, test_size=test_size)
    
    for fd, img, y in zip(['train', 'test'], (images_train, images_test), (y_train, y_test)):
        for lable in np.unique(y):
            copy_files_to_folder(img[y == lable], os.path.join(folder, fd, lable))
        
        

In [299]:
FOLDER = 'new_224_112_filter_1'
create_Folders_for_train_test(FOLDER)

In [293]:
def get_image_size(folder, add_folder='train'):
    """
    return the size of images in the folder
    """
    try:
        if add_folder:
            path = os.path.join(os.getcwd(), folder, add_folder)
        else:
            path = os.path.join(os.getcwd(), folder)
        
        while os.path.isdir(path):
            filename = os.listdir(path)[0]
            path = os.path.join(path, filename)
            print path
            
    except OSError as e:
        print e
        return

    return imageio.imread(path).shape

In [300]:
size = get_image_size(FOLDER)

/home/oleg/EnkeSystem/new_224_112_filter_1/train/video_bad
/home/oleg/EnkeSystem/new_224_112_filter_1/train/video_bad/08841_image.jpg


In [301]:
len(size), size

(2, (112, 224))

In [297]:
#libraries
import tensorflow as tf
import tflearn
tf.__version__

/home/oleg/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.10.0'

### Train test data sets

In [302]:
# gerator of the train set
datagen_tr = tf.keras.preprocessing.image.ImageDataGenerator(
    #featurewise_center=True,
    #samplewise_center=True,
    #featurewise_std_normalization=True,
    #samplewise_std_normalization=True,
    #rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    #rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
    )

In [303]:
os.listdir(os.path.join(os.getcwd(), FOLDER, 'train'))

['video_bad', 'video_good']

In [10]:
os.path.join(os.getcwd(), 'new_224_112_filter', 'train')

'/home/oleg/EnkeSystem/new_224_112_filter/train'

In [304]:
# train generator
train_generator = datagen_tr.flow_from_directory(
    os.path.join(os.getcwd(), FOLDER, 'train'),
    
    target_size=size,
    color_mode='grayscale',
    classes=None,
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    interpolation='nearest'
)

Found 22671 images belonging to 2 classes.


In [305]:
# generator of the test set
datagen_ts = tf.keras.preprocessing.image.ImageDataGenerator(
    #featurewise_center=True,
    #samplewise_center=True,
    #featurewise_std_normalization=True,
    #samplewise_std_normalization=True,
    #rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    #rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
    )

In [306]:
# test generator
test_generator = datagen_ts.flow_from_directory(
    os.path.join(os.getcwd(), FOLDER, 'test'),
    
    target_size=size,
    color_mode='grayscale',
    classes=None,
    class_mode='binary',
    batch_size=16,
    shuffle=True,
    seed=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    interpolation='nearest'
)

Found 4528 images belonging to 2 classes.


### Network building

In [311]:

#num_classes = 1
epochs = 100

model = tf.keras.Sequential()
#model.add(tf.keras.layers.BatchNormalization( input_shape=(112,224,1)))
#model.add(tf.keras.layers.Conv2D(2, (3,3), activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(2, (3,3), 
                 activation='relu',
                padding='same',
                input_shape=(112,224,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(4, 3, activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
#model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
#model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(512, activation='relu'))#
#model.add(tf.keras.layers.Dropout(0.5))
#model.add(tf.keras.layers.Dense(122, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 112, 224, 2)       20        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 112, 2)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 112, 4)        76        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 56, 4)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 56, 8)         296       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 28, 8)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 28, 8)         584       
__________

In [312]:
adam = tf.keras.optimizers.Adam(lr=1e-3, )

In [313]:
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [314]:
callbacks = [ tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0)
             #, ModelCheckpoint('video_3_512_VGG_no_drop.h5', monitor='val_loss', save_best_only=True, verbose=0) 
            ]
with tf.device('/gpu:0'):
    model.fit_generator(train_generator, workers=-1, 
                    steps_per_epoch=400, 
                    epochs=50, 
                    verbose=2,
                    #callbacks=callbacks,
                    use_multiprocessing=True,
                    validation_data=test_generator,
                    validation_steps=100
                     )

Epoch 1/50
 - 57s - loss: 0.8640 - acc: 0.5788 - val_loss: 0.6888 - val_acc: 0.5250
Epoch 2/50
 - 57s - loss: 0.6611 - acc: 0.6081 - val_loss: 0.6837 - val_acc: 0.5644
Epoch 3/50
 - 56s - loss: 0.6309 - acc: 0.6472 - val_loss: 0.6208 - val_acc: 0.6381
Epoch 4/50
 - 57s - loss: 0.5961 - acc: 0.6748 - val_loss: 0.5712 - val_acc: 0.7081
Epoch 5/50
 - 57s - loss: 0.5466 - acc: 0.7262 - val_loss: 0.5304 - val_acc: 0.7488
Epoch 6/50
 - 57s - loss: 0.5114 - acc: 0.7576 - val_loss: 0.4940 - val_acc: 0.7588
Epoch 7/50
 - 59s - loss: 0.4935 - acc: 0.7670 - val_loss: 0.4708 - val_acc: 0.7837
Epoch 8/50
 - 58s - loss: 0.4799 - acc: 0.7695 - val_loss: 0.4776 - val_acc: 0.7688
Epoch 9/50
 - 57s - loss: 0.4566 - acc: 0.7862 - val_loss: 0.4508 - val_acc: 0.7963
Epoch 10/50
 - 57s - loss: 0.4481 - acc: 0.7907 - val_loss: 0.4465 - val_acc: 0.7925
Epoch 11/50
 - 58s - loss: 0.4294 - acc: 0.8002 - val_loss: 0.4076 - val_acc: 0.8150
Epoch 12/50
 - 57s - loss: 0.4155 - acc: 0.8060 - val_loss: 0.4133 - val_a

In [315]:
model.save('cnn_model.h5')

### Network building 2

In [323]:
#num_classes = 1
epochs = 100

model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=(112,224,1)))
model.add(tf.keras.layers.Conv2D(2, (3,3), activation='relu',padding='same'))
#model.add(tf.keras.layers.Conv2D(2, (3,3), 
#                 activation='relu',
#                padding='same',
#                input_shape=(112,224,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(4, 3, activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
#model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
#model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(512, activation='relu'))#
#model.add(tf.keras.layers.Dropout(0.5))
#model.add(tf.keras.layers.Dense(122, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(44, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 112, 224, 1)       4         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 112, 224, 2)       20        
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 56, 112, 2)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 56, 112, 4)        76        
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 28, 56, 4)         0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 28, 56, 8)         296       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 14, 28, 8)         0         
__________

In [324]:
adam = tf.keras.optimizers.Adam(lr=1e-3, )

In [325]:
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [326]:
callbacks = [ tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0)
             #, ModelCheckpoint('video_3_512_VGG_no_drop.h5', monitor='val_loss', save_best_only=True, verbose=0) 
            ]
with tf.device('/gpu:0'):
    model.fit_generator(train_generator, workers=-1, 
                    steps_per_epoch=400, 
                    epochs=50, 
                    verbose=2,
                    #callbacks=callbacks,
                    use_multiprocessing=True,
                    validation_data=test_generator,
                    validation_steps=100
                     )

Epoch 1/50
 - 69s - loss: 0.6707 - acc: 0.6020 - val_loss: 0.6891 - val_acc: 0.5513
Epoch 2/50
 - 68s - loss: 0.5749 - acc: 0.6964 - val_loss: 0.5057 - val_acc: 0.7569
Epoch 3/50
 - 69s - loss: 0.4918 - acc: 0.7681 - val_loss: 0.4483 - val_acc: 0.7944
Epoch 4/50
 - 68s - loss: 0.4490 - acc: 0.7880 - val_loss: 0.4467 - val_acc: 0.7863
Epoch 5/50
 - 68s - loss: 0.4356 - acc: 0.7992 - val_loss: 0.4094 - val_acc: 0.8131
Epoch 6/50
 - 67s - loss: 0.4131 - acc: 0.8094 - val_loss: 0.3946 - val_acc: 0.8219
Epoch 7/50
 - 68s - loss: 0.3997 - acc: 0.8209 - val_loss: 0.3798 - val_acc: 0.8363
Epoch 8/50
 - 67s - loss: 0.3947 - acc: 0.8187 - val_loss: 0.3932 - val_acc: 0.8287
Epoch 9/50
 - 67s - loss: 0.3911 - acc: 0.8231 - val_loss: 0.3901 - val_acc: 0.8263
Epoch 10/50
 - 68s - loss: 0.3820 - acc: 0.8281 - val_loss: 0.3567 - val_acc: 0.8450
Epoch 11/50
 - 69s - loss: 0.3721 - acc: 0.8341 - val_loss: 0.3591 - val_acc: 0.8350
Epoch 12/50
 - 69s - loss: 0.3692 - acc: 0.8332 - val_loss: 0.3652 - val_a

In [327]:
model.save('cnn_model2.h5')